In [1]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
import glob
import pandas as pd
#import PySimpleGUI as sg
from collections import defaultdict
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
import numpy as np
from gensim import models

!apt-get install -y xvfb # Install X Virtual Frame Buffer
import os
os.system('Xvfb :1 -screen 0 1600x1200x16  &')    # create virtual display with size 1600x1200 and 16 bit color. Color can be changed to 24 or 8
os.environ['DISPLAY']=':1.0'    # tell X clients to use our virtual DISPLAY :1.0

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Reading package lists... Done
Building dependency tree       
Reading state information... Done
xvfb is already the newest version (2:1.19.6-1ubuntu4.9).
0 upgraded, 0 newly installed, 0 to remove and 40 not upgraded.


In [2]:
dataf = pd.read_csv('/content/drive/MyDrive/information retrieval/profileTables.csv')

print(dataf.shape)

(19475, 4)


In [3]:
datafText= dataf['Title']
datafText

0        Adaptive Filtering-Based Pseudo Open-Loop Thre...
1        Coordinate Transformation-Free Observer-based ...
2        Demodulation Type Single-Phase PLL with DC Off...
3        Frequency Adaptive Parameter Estimation of Unb...
4        Gain Normalized Adaptive Observer For Three-Ph...
                               ...                        
19470    A Sensitivity Approach for Eliminating Clashes...
19471    Using parametric sensitivity analysis to detec...
19472               Design Intent for CAD model assemblies
19473    Implementing a maths support system for first-...
19474    Sensitivity approach for automatically elimina...
Name: Title, Length: 19475, dtype: object

In [4]:
#Eliminating stopwords 
#this will contain a list of all the words in the corpus

datafText_words = []
# Tokenize a paragraph into sentences and each sentence in to
# words
for c in datafText:
    for sent in sent_tokenize(c):
      word_tokens = word_tokenize(sent)
      datafText_words += word_tokens

len(datafText)

19475

In [5]:
#inspecting the words in the datafText a bit
datafText_words.index('the'), datafText_words.index('The')

(90, 724)

In [6]:
datafText_words.index('so'), datafText_words.index('So')

(25338, 51760)

In [7]:
lower_case_words = set([ x.lower() for x in datafText_words])
len(lower_case_words)

23516

In [8]:
#remove stopwords
stwords = set(stopwords.words('english'))
#using set difference to calculate stopwords from our words

stopfree_words = lower_case_words - stwords
len(stopfree_words)




23383

In [9]:
#Stemming
from nltk.stem import snowball

stemmer = snowball.SnowballStemmer('english')
stemmed_words = set([stemmer.stem(x) for x in stopfree_words])
len(stemmed_words)


18003

In [10]:
# Lets look at some of our words
list(stemmed_words)[-10:]

['clockwork',
 'project-bas',
 'alkalin',
 'plastic',
 'encyclopaedia',
 'wick',
 'intuit',
 'nb-dope',
 'ibigw',
 'pneumovirus']

In [11]:
#constructing the index
# Our index is a map of word -> documents it is found in
inverted_index = defaultdict(set)

# We maintain the reference to the document by its index in the corpus list
for docid, c in enumerate(datafText):
    for sent in sent_tokenize(c):
        for word in word_tokenize(sent):
            word_lower = word.lower()
            if word_lower not in stwords:
                word_stem = stemmer.stem(word_lower)
                # We add the document to the set againt the word in our
                # index
                inverted_index[word_stem].add(docid)

len(inverted_index.keys())

18003

In [12]:
#searching using the index

def operationalSearch(query):
    matched_documents = set()
    for word in word_tokenize(query):
        word_lower = word.lower()
        if word_lower not in stwords:
            word_stem = stemmer.stem(word_lower)
            matches = inverted_index.get(word_stem)
            if matches:
                # The operator |= is a short hand for set union
                matched_documents |= matches
    return matched_documents


In [13]:
operationalSearch("phase")

#the ouput is where the word "phase" appears in documents

{13,
 42,
 1101,
 1113,
 1119,
 1122,
 1925,
 1967,
 1969,
 2271,
 2273,
 2636,
 3139,
 3141,
 3149,
 3150,
 3156,
 3160,
 3166,
 3167,
 3169,
 3173,
 3174,
 3181,
 3196,
 3207,
 3213,
 3226,
 3227,
 3335,
 3337,
 3386,
 3707,
 3970,
 4138,
 4159,
 4185,
 4394,
 5136,
 5142,
 5238,
 5465,
 5552,
 5609,
 5634,
 5635,
 5658,
 5664,
 5775,
 5785,
 5798,
 5805,
 5808,
 5812,
 5975,
 5976,
 5981,
 5987,
 6012,
 6400,
 6423,
 6462,
 6594,
 6596,
 6951,
 7149,
 7495,
 7776,
 7943,
 7950,
 8198,
 8220,
 8223,
 8978,
 8983,
 8984,
 8987,
 8989,
 8994,
 8998,
 8999,
 9041,
 9050,
 9070,
 9077,
 9314,
 9364,
 9562,
 9626,
 9648,
 9978,
 10083,
 10805,
 10948,
 11364,
 11557,
 11933,
 12153,
 12165,
 12351,
 12489,
 13222,
 13229,
 13231,
 13233,
 13414,
 14058,
 14132,
 14253,
 15242,
 15243,
 15864,
 16029,
 16031,
 16059,
 16546,
 17014,
 17019,
 17031,
 17072,
 17295,
 17780,
 17928,
 17929,
 17930,
 17934,
 18079,
 18475,
 18477,
 18500,
 18503,
 18504}

In [14]:
operationalSearch("blue")

{6582, 9716, 12731, 14140}

In [15]:
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer 
import string

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [16]:
record = dataf.loc[0,:].copy()

In [17]:
record

Name                                                      Hafiz Ahmed
profile_URL         https://pureportal.coventry.ac.uk/en/persons/a...
publications_url    https://pureportal.coventry.ac.uk/en/persons/a...
Title               Adaptive Filtering-Based Pseudo Open-Loop Thre...
Name: 0, dtype: object

In [18]:
def Operationalstring(content):
  content = content.lower() #to lowercase
  content = content.translate(str.maketrans('', '', string.punctuation)) #strip punctuation
  return content

In [19]:

Operationalstring(record.Title)

'adaptive filteringbased pseudo openloop threephase gridsynchronization technique'

In [20]:

def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

In [21]:
print("Apple: {}\n Run: {}\n Happy: {}" .format(get_wordnet_pos("apple"), get_wordnet_pos("run"), get_wordnet_pos("happy")))

Apple: n
 Run: v
 Happy: a


In [22]:
stop = stopwords.words('english')
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [23]:
#################################################################################################

In [24]:
lem = WordNetLemmatizer()

def stopLemmatize(doc):
    tokens = nltk.word_tokenize(doc)
    tmp = ""
    for w in tokens:
        if w not in stop:
            tmp += lem.lemmatize(w, get_wordnet_pos(w)) + " "
    return tmp

In [25]:
stopLemmatize(doc = record.Title)

'Adaptive Filtering-Based Pseudo Open-Loop Three-Phase Grid-Synchronization Technique '

In [26]:
def Operationalstring(content):
  content = content.lower() #to lowercase
  content = content.translate(str.maketrans('', '', string.punctuation)) #strip punctuation
  content = stopLemmatize(content)
  return content

In [27]:
%time Operationalstring(record.Title)

CPU times: user 2.26 ms, sys: 1.97 ms, total: 4.23 ms
Wall time: 4.08 ms


'adaptive filteringbased pseudo openloop threephase gridsynchronization technique '

In [28]:
meta = dataf.copy()

In [29]:
def convertDataf(dataf):
  dataf['Title'] = dataf['Title'].apply(Operationalstring)
  dataf['Name'] = dataf['Name'].apply(Operationalstring)

In [30]:
%time convertDataf(meta)

CPU times: user 39.8 s, sys: 1.75 s, total: 41.6 s
Wall time: 41.6 s


In [31]:
meta.head(5)

,Name,profile_URL,publications_url,Title
0,hafiz ahmed,https://pureportal.coventry.ac.uk/en/persons/a...,https://pureportal.coventry.ac.uk/en/persons/a...,adaptive filteringbased pseudo openloop threep...
1,hafiz ahmed,https://pureportal.coventry.ac.uk/en/persons/a...,https://pureportal.coventry.ac.uk/en/persons/a...,coordinate transformationfree observerbased ad...
2,hafiz ahmed,https://pureportal.coventry.ac.uk/en/persons/a...,https://pureportal.coventry.ac.uk/en/persons/a...,demodulation type singlephase pll dc offset re...
3,hafiz ahmed,https://pureportal.coventry.ac.uk/en/persons/a...,https://pureportal.coventry.ac.uk/en/persons/a...,frequency adaptive parameter estimation unbala...
4,hafiz ahmed,https://pureportal.coventry.ac.uk/en/persons/a...,https://pureportal.coventry.ac.uk/en/persons/a...,gain normalize adaptive observer threephase sy...


In [32]:
meta['Text'] = meta['Title'] + " " + meta['Name']
drop_cols = ['profile_URL', 'publications_url']
meta = meta.drop(drop_cols, axis=1)

In [33]:
meta.head(5)

,Name,Title,Text
0,hafiz ahmed,adaptive filteringbased pseudo openloop threep...,adaptive filteringbased pseudo openloop threep...
1,hafiz ahmed,coordinate transformationfree observerbased ad...,coordinate transformationfree observerbased ad...
2,hafiz ahmed,demodulation type singlephase pll dc offset re...,demodulation type singlephase pll dc offset re...
3,hafiz ahmed,frequency adaptive parameter estimation unbala...,frequency adaptive parameter estimation unbala...
4,hafiz ahmed,gain normalize adaptive observer threephase sy...,gain normalize adaptive observer threephase sy...


In [34]:

def convertDataf(dataf):
  dataf = dataf
  dataf['Title'] = dataf['Title'].apply(Operationalstring)
  dataf['Name'] = dataf['Name'].apply(Operationalstring)
  dataf['text'] = dataf['Title'] + " " + dataf['Name']
  drop_cols = ['profile_URL', 'publications_url']
  dataf = dataf.drop(drop_cols, axis=1)
  return dataf

In [35]:
#build index

In [36]:
record = meta.loc[0,:].copy()
print(record)
testIndex= {}

Name                                          hafiz ahmed 
Title    adaptive filteringbased pseudo openloop threep...
Text     adaptive filteringbased pseudo openloop threep...
Name: 0, dtype: object


In [37]:
words = record.Text.split()
Text = record.Text

In [38]:
word = words[0]
wordSample = {word: [Text]}
print(wordSample)#

{'adaptive': ['adaptive filteringbased pseudo openloop threephase gridsynchronization technique  hafiz ahmed ']}


In [39]:
for word in words:
  if word in testIndex.keys():
    if Text not in testIndex[word]:
      testIndex[word].append(word)
  else:
    testIndex[word] = [word]

In [40]:
print(testIndex)

{'adaptive': ['adaptive'], 'filteringbased': ['filteringbased'], 'pseudo': ['pseudo'], 'openloop': ['openloop'], 'threephase': ['threephase'], 'gridsynchronization': ['gridsynchronization'], 'technique': ['technique'], 'hafiz': ['hafiz'], 'ahmed': ['ahmed']}


In [41]:

def index_it(record, index):
  words = record.Title.split()
  Title = record.Title
  for word in words:
    if word in index.keys():
      if Title not in index[word]:
        index[word].append(Title)
    else:
      index[word] = [Title]
  return index

In [42]:
ind = index_it(record=record, index= {})
print(ind)
print(len(ind))

{'adaptive': ['adaptive filteringbased pseudo openloop threephase gridsynchronization technique '], 'filteringbased': ['adaptive filteringbased pseudo openloop threephase gridsynchronization technique '], 'pseudo': ['adaptive filteringbased pseudo openloop threephase gridsynchronization technique '], 'openloop': ['adaptive filteringbased pseudo openloop threephase gridsynchronization technique '], 'threephase': ['adaptive filteringbased pseudo openloop threephase gridsynchronization technique '], 'gridsynchronization': ['adaptive filteringbased pseudo openloop threephase gridsynchronization technique '], 'technique': ['adaptive filteringbased pseudo openloop threephase gridsynchronization technique ']}
7


In [43]:
#iterate over all records in the database with scraped publication titles, process them append to index.
def indexAll(meta, index):
  for i in range(len(meta)):
    record = meta.loc[i,:]
    index = index_it(record = record, index = index)
  return index

In [44]:
index = indexAll(meta, index = {})
len(index)

19835

In [45]:
def buildIndex(meta, index):
    to_add = convertDataf(dataf)
    index = indexAll(meta = to_add, index = index)
    return index

In [46]:
idx = buildIndex(meta = dataf, index = {})

In [47]:
len(idx)

19835

In [48]:
import json

with open("/content/drive/MyDrive/information retrieval/7071CEM Information Retrieval/CW/New folder/pureportal_survey_json.json", 'w') as fp:
    json.dump(idx, fp, sort_keys=True, indent=4)

In [49]:
with open('/content/drive/MyDrive/information retrieval/7071CEM Information Retrieval/CW/New folder/pureportal_survey_json.json', 'r') as f:
    pureportal_survey_json = json.load(f)

In [50]:
word2vec = pureportal_survey_json

In [51]:
print(words)

['adaptive', 'filteringbased', 'pseudo', 'openloop', 'threephase', 'gridsynchronization', 'technique', 'hafiz', 'ahmed']


In [52]:

# Python program to generate word vectors using Word2Vec
  
# importing all necessary modules
from nltk.tokenize import sent_tokenize, word_tokenize
import gensim
import warnings
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [53]:
#################################################################################
#query processor

In [54]:
check = "filtering based"

In [55]:
print("User query: {}." .format(check))
normTest = Operationalstring(check)
print("Normalized query: {}." .format(normTest))

User query: filtering based.
Normalized query: filter base .


In [56]:
checkSplit = normTest.split()

In [57]:
def operationQuery(query):
  norm = Operationalstring(query)
  return norm.split()

In [58]:
#retreive from index
retrieved = []
for word in checkSplit:
  if word in index.keys():
    retrieved.append(index[word])

In [59]:
#looking for intersection of all posting lists
def intersectLists(lists):
  intersect = list(set.intersection(*map(set, lists)))
  intersect.sort()
  return intersect

In [60]:
outcome = intersectLists(retrieved)
print(outcome)

['information fusion algorithm vehicle state estimation base extend kalman filter ', 'numerical achieve extend kalman filter state observer design base vehicle model contain unitire model ', 'state charge estimation lithiumion battery base intelligent adaptive unscented kalman filter ', 'vehicle drive state estimation base extend kalman filter ']


In [61]:
def googleishSearch(query, index=idx):
 splitQuery = operationQuery(query)
 retrieved = []
 for term in splitQuery:
  if term in index.keys():
    retrieved.append(index[term])
 if len(retrieved)>0:
  outcome = intersectLists(retrieved)
 else:
    outcome = [0]
 return outcome

In [62]:
outcome_IDs = googleishSearch("historical", index)
print(outcome_IDs)
print(len(outcome_IDs))

['advertising public memory social cultural historical perspective ghost sign ', 'britain islamic diaspora historical précis neoteric ethnography british muslim ', 'decommonising mind historical impact british imperialism indigenous tenure system selfunderstanding highland island scotland ', 'enlightenment historical write expulsion england ’ jew ', 'four eponym sunni islamic jurisprudence examination historical development dominant madhâhib polemic ijtihâd versus taqlîd ', 'hisql frontend query system historical database ', 'historical institutionalist perspective persistence state control financial sector reform insightful case myanmar ', 'historical pollution human right violation role criminal law ', 'historical pollution longterm liability global challenge need international approach ', 'historical write britain 16881830 vision history ', 'impact select historical event incomebased distribution terrorism rise fundamentalist terrorism iraq war ', 'international law islam historical

In [63]:
#Retrieve meta-data

In [64]:
#Query from database to get only rows of outcome IDs

def connectIDdataf(retrieved_id, dataf):
    return dataf[dataf.Title.isin(retrieved_id)].reset_index(drop=True)

In [65]:
result_meta = connectIDdataf(outcome_IDs, meta)
result_meta.head(5)

,Name,Title,Text
0,rachid anane,hisql frontend query system historical database,hisql frontend query system historical databas...
1,kristin aune,religion high education europe north america h...,religion high education europe north america h...
2,megan crawford,mapping field 40 year historical review,mapping field 40 year historical review megan...
3,benjamin dew,enlightenment historical write expulsion engla...,enlightenment historical write expulsion engla...
4,benjamin dew,historical write britain 16881830 vision history,historical write britain 16881830 vision histo...


In [71]:
query = input("Search for:")
googleishSearch(query)

Search for:filtering


['384 tmacs fir filter artix7 fpga use prism signal processing ',
 'bearing natural degradation detection gearbox comparative study effectiveness adaptive filter algorithm spectral kurtosis ',
 'box regularize particle filter state estimation severely ambiguous nonlinear measurement ',
 'building billion tap filter ',
 'comparative study adaptive filter detect naturally degrade bearing within gearbox ',
 'comparative study effectiveness adaptive filter algorithm spectral kurtosis linear prediction detection naturally degrade bearing gearbox ',
 'conditional filter reduction search space genetic algorithm optimisation sensor actuator location active vibration control ',
 'critical comparison attitude estimation gaussian approximate filter coordinatefree dual optimal control ',
 'dual extend kalman filter combine estimation vehicle state road friction ',
 'effect gridfilter width definition implicitly filter large eddy simulation use openfoam ',
 'effectiveness adaptive filter algorithm 

In [67]:
# Interface Code #
##
val_list = []
sg.theme_input_text_color('#000000')

layout = [[sg.Text("Type a query")],
          [sg.Input(key='query')],
          [sg.Button('Search')],
          [sg.Text(size=(20,1))],
          [sg.Table(values=val_list, headings=["Name", "Title", "publications_url"], background_color='gray',
                    auto_size_columns=False, display_row_numbers=False,justification='left', num_rows=15, col_widths = [5,20,40,100,10,20],
                    key='TABLE', row_height=25, enable_events=False)],
        [sg.Button('Exit')]]

window = sg.Window('Search Engine', layout)

while True:
    event, values = window.read()

    if event == sg.WINDOW_CLOSED or event == 'Exit':
        break
    
    query = values['query']

    try:
        idx, results = search(index, data, 30, query, "Name", "Title")
        print(results)

        window['TABLE'].update(values = results)
    except:
        print("error")

window.close()

NameError: ignored

In [ ]:
!pip install PySimpleGUI